### Dataset Preparation

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras import regularizers
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn import preprocessing 
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import seaborn as sns
sns.set(style="whitegrid")
np.random.seed(203)

data = pd.read_csv("./creditcard.csv")
data.head()

In [ ]:
data["Time"] = data["Time"].apply(lambda x : x / 3600 % 24) # 비율을 나타내고 싶었던 것 같음
data.head()

데이터셋 : 총 31개의 변수

Time : 첫번째 transaction으로부터 경과된 시간(초)
V1 ~ V28 : PCA 변환된 변수(28개)
Amount : 거래 금액
Class : Target 변수(0: 정상, 1: 비정상)

총 284,807건의 거래내역, 이 중 사기 거래(Fraud Transaction)는 492건

In [ ]:
colors = ["#0101DF", "#DF0101"]
LABELS = ["Non-Fraud", "Fraud"]
sns.countplot('Class', data=data, palette=colors)
plt.title('Class Distribution \n (0 : Non-Fraud / 1 : Fraud)', fontsize=13)
plt.xticks(range(2), LABELS)
plt.xlabel("Class")
plt.ylabel("Frequency");

In [ ]:
vc = data['Class'].value_counts().to_frame().reset_index()
vc['percent'] = vc["Class"].apply(lambda x : round(100*float(x) / len(data), 2))
vc = vc.rename(columns = {"index" : "Target", "Class" : "Count"})
vc

In [ ]:
data.describe()

In [ ]:
# non-fraud 거래내역의 1,000행만 사용

non_fraud = data[data['Class'] == 0].sample(1000)
fraud = data[data['Class'] == 1]

df = non_fraud.append(fraud).sample(frac=1).reset_index(drop=True)
X = df.drop(['Class'], axis = 1).values
Y = df["Class"].values

### Visualize Fraud and Non-Fraud Transactions

이상치에 대한 기존 데이터의 패턴 파악을 위해 클러스터링을 활용한 거래의 특성 시각화.
패턴 존재 여부 파악을 위한 t-SNE 활용으로 거래의 특성 시각화
-> 2차원으로 차원 축소하여, 고차원 데이터의 시각화에 사용. 
-> 각 데이터 포인트 주변으로 유사도를 계산하여, 2차원에서 원본 특성 공간에서 데이터 클러스터링 진행 (이웃 데이터 포인트에 대한 정보 보존 최적화)

또 다른 방식, PCA (linear한 방법으로 클러스터링) : 공분산 행렬에서 고유벡터를 계산하여 진행 

### t-SNE 

In [ ]:
def tsne_plot(x1, y1, name="graph.png"):
    tsne = TSNE(n_components=2, random_state=0)
    X_t = tsne.fit_transform(x1)

    plt.figure(figsize=(12, 8))
    plt.scatter(X_t[np.where(y1 == 0), 0], X_t[np.where(y1 == 0), 1], marker='o', color='g', alpha=0.8, label='Non Fraud')
    plt.scatter(X_t[np.where(y1 == 1), 0], X_t[np.where(y1 == 1), 1], marker='o', color='r', alpha=0.8, label='Fraud')

    plt.legend(loc='best');
    plt.show();

In [ ]:
tsne_plot(X, Y, "original.png")

결론 : 두 거래의 뚜렷한 특징을 찾기 어려움

### Auto-encoder 

auto-encoder를 활용하여, 입력 데이터와 복원덴 데이터 사이의 차이를 계산하여 이상치 탐지 
- 가정1 : 데이터가 잘 복원될 경우, 데이터 특성 공간이 잘 파악되었다는 것이며, 데이터 특성 공간이 잘 파악되었다는 것은, 정상 관측치일 가능성이 높다.
(즉, 입력된 데이터 특성을 요약하고 복원하는 과정을 포함한 auto-encoder로 자기지도학습을 진행하여, 재구축 오차가 적은 경우를 정상 관측지 / 오차가 큰 경우 이상치 (사기 데이터셋) 로 생각)
- 가정에 대한 근거 : 사기 데이터셋이 압도적으로 적은 경우이므로, 비사기 데이터셋을 대상으로 복원 프로세스를 대부분 진행하므로 사기 데이터셋의 재구성은 학습이 거의 안되어 복원이 잘 되지 않을 것

(모든 베이스 가정 : 사기 데이터셋과 비사기 데이터셋 간 차이가 존재할 것이다 => 정규분포에서 이것이 잘 드러나면 좋겠으나 적은 데이터셋으로 인해 잘 구분되지 않는 경향성을 보완하고자 auto-encoder 방식 활용함)

In [ ]:
## input layer 
input_layer = Input(shape=(X.shape[1],))

## encoding part
encoded = Dense(100, activation='tanh', activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoded = Dense(50, activation='relu')(encoded)

## decoding part
decoded = Dense(50, activation='tanh')(encoded)
decoded = Dense(100, activation='tanh')(decoded)

## output layer
output_layer = Dense(X.shape[1], activation='relu')(decoded)

# autoencoder 
autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer="adadelta", loss="mse")

In [ ]:
# 각 변수의 범위가 다른 것을 스케일링하여 전처리 : 2,000행의 비사기 케이스만 활용하여 학습 (적은 데이터셋으로도 구분 가능)

x = data.drop(["Class"], axis=1)
y = data["Class"].values

x_scale = preprocessing.MinMaxScaler().fit_transform(x.values)
x_norm, x_fraud = x_scale[y == 0], x_scale[y == 1]

In [ ]:
autoencoder.fit(x_norm[0:2000], x_norm[0:2000], batch_size = 256, epochs = 10, shuffle = True, validation_split = 0.20);

### latent representation

In [ ]:
hidden_representation = Sequential()
hidden_representation.add(autoencoder.layers[0])
hidden_representation.add(autoencoder.layers[1])
hidden_representation.add(autoencoder.layers[2])

In [ ]:
norm_hid_rep = hidden_representation.predict(x_norm[:3000])
fraud_hid_rep = hidden_representation.predict(x_fraud)

# 입력에 대한 잠재 변수 표현. sequential layer 포함하는 네트워크를 생성해 input에 대한 예측을 통해 잠재 변수를 추출.
# 즉, fraud와 non-fraud에 대한 두 숨겨진 표현 (잠재 표현)을 생성함

### Visualize

In [ ]:
rep_x = np.append(norm_hid_rep, fraud_hid_rep, axis = 0)
y_n = np.zeros(norm_hid_rep.shape[0])
y_f = np.ones(fraud_hid_rep.shape[0])
rep_y = np.append(y_n, y_f)
tsne_plot(rep_x, rep_y, "latent_representation.png")

시각화 한 결과, 잠재변수를 활용할 경우 훨씬 classifier에 적합한 형태를 만들 수 있음을 보임.
이렇게 할 경우 간단한 linear classifier 만으로도 변별이 가능함. 

### Simple linear classifier : using Ligistic Regression

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(rep_x, rep_y, test_size=0.25)
clf = LogisticRegression(solver="lbfgs").fit(train_x, train_y)
pred_y = clf.predict(val_x)

print ("")
print ("Classification Report: ")
print (classification_report(val_y, pred_y))

print ("")
print('Logistic Regression Accuracy Score: ', round(accuracy_score(val_y, pred_y) * 100, 3).astype(str) + '%')

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_pred=pred_y, y_true=val_y)
cmp = ConfusionMatrixDisplay(cm)
cmp.plot(cmap=plt.cm.Blues)

test 873건 (744 + 129) 
성능 : 129건의 fraud중 114건은 맞추고, 17건은 틀림

### what can i do..?

이런 식으로, 뭔가 특징이 불분명해서 내가 임의로 조작해야만 결과가 잘 나올 수 있는 사기/비사기 데이터셋에 대해,
문득 드는 생각 : 모델의 robust를 측정하기 위한 작업이 유의미한 작업인가?

- 원래 하려던 것
1. 사기 데이터셋에서 값 변형을 일부만 주어 비사기 데이터셋으로 분류하는지 확인하기 
2. 데이터셋이 없다는 가정 하에서, 그냥 랜덤한 값들을 쭉 던지며 이상치/비이상치 값을 구분하는 선형 분류 함수의 라인 찾기

추가작업 : 지도학습 말고 다른 학습 방식 없는 지 찾아보기!